In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, Legend
from bokeh.layouts import widgetbox, column
from bokeh.plotting import figure, curdoc
from bokeh.transform import factor_cmap
from bokeh.palettes import RdYlBu, Category10, Viridis6 
from IPython.core.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sn

pd.set_option("display.max_rows", 8)
pd.set_option("display.max_columns", 20)

In [2]:
cases_global = pd.read_csv('../data/Global_all_vars.csv').iloc[:,1:]
cases_global['Date'] = pd.to_datetime(cases_global['Date'], cache=True)

In [3]:
# corr_global = cases_global.drop('FIPS',axis=1).corr()
# sn.heatmap(corr_global, annot=True, annot_kws={"size":15})
# sn.set_context("poster",font_scale=1.5)
# sn.set(rc={'figure.figsize':(15,15)})
display(HTML("<p style='font-size: 15px'>Global Correlation Plot</p>"))

![img](assets/img/Seaborn/Corr_Plot_Global.png)

In [4]:
hover = HoverTool(tooltips=[
    ("Country/Region", "@Country_Region"),
    ("Area per sq km", "@{Area.sq.km}{0.00}"),
    ("Total_Cases", "@Total_Cases")
    ])

p = figure(title = "Area per square kilometer vs Total Cases", plot_height=500, plot_width=750,
           tools=[hover,"pan,reset,wheel_zoom"])

p.circle('Area.sq.km','Total_Cases',
         fill_alpha=0.5, size=10, color='blue',#fill_color=colors, line_color=colors,
         source=ColumnDataSource(cases_global[cases_global['Date'] == pd.Timestamp("2020-06-30")]))
p.xaxis.axis_label = 'Area per square kilometer (thousands)'
p.yaxis.axis_label = 'Total COVID-19 Cases (06/30/2020)'
# p.legend.title = 'Avg Person Per Household'
# p.legend.location = "top_left"
# p.legend.orientation = "horizontal"

output_file("../assets/img/Bokeh/Global_Area_Temp.html")

show(p)

<iframe src="assets/img/Bokeh/Global_Area_Temp.html"
    sandbox="allow-same-origin allow-scripts"
    width="100%"
    height="525"
    scrolling="no"
    seamless="seamless"
    frameborder="0">
</iframe>

In [5]:
def assign_class_pop(row):
    if row['PopTotal'] < 80000:
        return "< 80,000"
    elif row['PopTotal'] >= 80000 and row['PopTotal'] <= 100000:
        return ">= 80,000 & <= 100,000"
    else:
        return "> 100,000"
    
cases_global['PopTotalFCT'] = cases_global.apply(assign_class_pop, axis=1)

hover = HoverTool(tooltips=[
    ("Country/Region", "@Country_Region"),
    ("Density", "@PopDensity{0.00}"),
    ("Population", "@PopTotal{0.00}"),
    ("Total_Cases", "@Total_Cases")
    ])

p = figure(title = "Population Density vs Total Cases", plot_height=500, plot_width=750,
           tools=[hover,"pan,reset,wheel_zoom"])

colors = factor_cmap('PopTotalFCT', palette=['#EC7063','#A569BD','#007BCC'],
                     factors=cases_global['PopTotalFCT'].unique()) 

p.circle('PopDensity','Total_Cases', legend='PopTotalFCT',
         fill_alpha=0.5, size=10, fill_color=colors, line_color=colors,
         source=ColumnDataSource(cases_global[cases_global['Date'] == pd.Timestamp("2020-06-30")]))
p.xaxis.axis_label = 'Population per square kilometer'
p.yaxis.axis_label = 'Total COVID-19 Cases (06/30/2020)'
p.legend.title = 'Total Population'
p.legend.location = "top_right"

cases_global = cases_global.drop('PopTotalFCT', axis=1)

output_file("../assets/img/Bokeh/Global_Density_Temp.html")

show(p)

<iframe src="assets/img/Bokeh/Global_Density_Temp.html"
    sandbox="allow-same-origin allow-scripts"
    width="100%"
    height="500"
    scrolling="no"
    seamless="seamless"
    frameborder="0">
</iframe>